In [ ]:
"""
Tumor Classsification using SqueezeNet
Author: Amruth Karun M V
Date: 12-Nov-2021
"""

import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Dense, Input, Dropout, Flatten, 
    Conv2D, MaxPooling2D, concatenate,
    GlobalAveragePooling2D, Activation
)

from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline

TRAIN_PATH = "../input/brain-tumor-mri-dataset/Training"
TEST_PATH = "../input/brain-tumor-mri-dataset/Testing"
CLASS_NAMES = ['Glioma', 'Meningioma', 'No-tumor', 'Pituitary']
EPOCHS = 100
BATCH_SIZE = 128
LEARNING_RATE = 0.001


def plot_sample_images():
    """
    Plots sample images for each class
    Arguments: None
    Returns: Plots sample data
    """
    
    plt.figure(figsize=(10, 10))
    sample_image_path = ['/glioma/Tr-gl_0010.jpg', '/meningioma/Tr-me_0010.jpg', 
                         '/notumor/Tr-no_0010.jpg', '/pituitary/Tr-pi_0010.jpg']
    for i in range(len(CLASS_NAMES)):
        ax = plt.subplot(2, 2, i + 1)
        img = cv2.imread(TRAIN_PATH + sample_image_path[i])
        img = cv2.resize(img, (128, 128))
        plt.imshow(img)
        plt.title(CLASS_NAMES[i])
    
    
def load_data(input_path, shuffle=False):
    """
    Loads input data fro directory
    Arguments:
        input_path -- input data path
        shuffle    -- whether data needs to be shuffled or not
    Returns: Data generator
    """
    
    data_generator = keras.preprocessing.image.ImageDataGenerator()
    data_generator = data_generator.flow_from_directory(directory=input_path, target_size=(224,224), 
                                                        shuffle=shuffle, class_mode= "categorical")
    
    return data_generator


def fire_module(x, fire_id, squeeze=16, expand=64):
    """
    Fire module for SqueezeNet
    Arguments:
        x        -- input
        fire_id  -- id for the module
        squeeze  -- No. of squeeze layer filters
        expand   -- No. of expand layer filters
    Returns: Concatenated output of fire_module
    """
    
    s_id = 'fire' + str(fire_id) + '/'
    
    x = Conv2D(squeeze, (1, 1), padding='valid', name=s_id + 'sq1x1')(x)
    x = Activation('relu', name=s_id + 'relu_sq1x1')(x)

    left = Conv2D(expand, (1, 1), padding='valid', name=s_id + 'exp1x1')(x)
    left = Activation('relu', name=s_id + 'relu_exp1x1')(left)

    right = Conv2D(expand, (3, 3), padding='same', name=s_id + 'exp3x3')(x)
    right = Activation('relu', name=s_id + 'relu_exp3x3')(right)

    x = concatenate([left, right], axis=3, name=s_id + 'concat')
    return x


def load_model():
    """
    Creates a keras SqueezeNet model
    Arguments: None
    Returns: SqueezeNet Model
    """
    
    img_input = Input(shape=(224, 224, 3))
    
    x = Conv2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(img_input)
    x = Activation('relu', name='relu_conv1')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)
    
    # Classification block
    x = Dropout(0.5, name='drop9')(x)
    x = Conv2D(4, (1, 1), padding='valid', name='conv10')(x)
    x = Activation('relu', name='relu_conv10')(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation('softmax', name='loss')(x)
    
    model = Model(img_input, x, name='SqueezeNet')
    
    model.summary()
   
    opt = Adam(learning_rate=LEARNING_RATE)
    model.compile(loss = keras.losses.categorical_crossentropy, optimizer=opt, metrics=['accuracy'])
    
    return model


def plot_curves(history):
    """
    Plots loss and accuracy and loss plots for
    training and validation datasets
    Arguments: 
        history -- training history
    Returns: None
    """
   
    plt.plot(history.history['loss'], label="Training loss")
    plt.plot(history.history['val_loss'], label="Validation loss")
    plt.legend()
    plt.title('Training Loss VS Validation Loss')
    plt.show()
    
    plt.plot(history.history['accuracy'], label="Training accuracy")
    plt.plot(history.history['val_accuracy'], label="Validation accuracy")
    plt.title('Training Accuracy VS Validation Accuracy')
    plt.legend()
    plt.show()
    

def evaluate_model(model, input_path):
    """
    Evaluates the model and displays
    the confusion matrix
    Arguments:
        model       -- trained model
        input_path  -- input data path
    Returns: Model score and confusion matrix
    """

    data_generator = load_data(input_path)
    predictions = model.predict(data_generator, BATCH_SIZE)
    y_pred = np.argmax(predictions, axis=1)
    y_true = data_generator.classes
    
    print("Score = ", model.evaluate(data_generator))
    print("Accuracy = ", metrics.accuracy_score(y_true, y_pred))
    cm = metrics.confusion_matrix(y_true, y_pred)
    metrics.ConfusionMatrixDisplay(cm, display_labels=CLASS_NAMES).plot(cmap=plt.cm.Blues,
                                                                       xticks_rotation='vertical')
    plt.show()

    
def train_model():
    """
    Trains SqueezeNet model and saves the 
    trained weights to an H5 file.
    Arguments: None
    Returns: None
    """
    
    train_generator = load_data(TRAIN_PATH, True)
    val_generator = load_data(TEST_PATH, True)
    
    # Loads SqueezeNet model
    model = load_model()
    earlystop = keras.callbacks.EarlyStopping(monitor='loss', min_delta=1e-11, patience=10)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, 
                                                   patience=6, verbose=1)
    model_callbacks = [earlystop, reduce_lr]
    
    history = model.fit(
        train_generator, 
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=val_generator,
        validation_steps=val_generator.samples//BATCH_SIZE,
        steps_per_epoch=train_generator.samples//BATCH_SIZE,
        callbacks=model_callbacks)
    
    plot_curves(history)
    model.save_weights("model_squeezenet.h5")
    print("Model saved successfully!")
    
    return model

In [ ]:
plot_sample_images()

In [ ]:
# Train the model
model = train_model()
print("Confusion matrix for train data: ")
evaluate_model(model, TRAIN_PATH)
print("Confusion matrix for val data: ")
evaluate_model(model, TEST_PATH)